In [1]:
import pandas as pd
import geopandas

In [2]:
vita_sites_df = pd.read_csv("./data/vita_providers_created_by_our_irs_vita_site_downloader_2022-03-24T15 48 05.984215Z.csv")

Per CFA email, we want only sites where `archived=False`.

In [3]:
vita_sites_df = vita_sites_df[vita_sites_df.archived == False]

In [4]:
points = geopandas.GeoSeries.from_wkb(vita_sites_df.coordinates)
vita_gdf = geopandas.GeoDataFrame(vita_sites_df, geometry=points)

In [5]:
vita_gdf.sample(40)

,id,appointment_info,archived,coordinates,created_at,dates,details,hours,irs_id,languages,last_scrape_id,name,updated_at,geometry
1336,2460,Required,False,0101000020E61000006823D74D299954C093C83EC8B20C...,2020-01-10T19:32:37.11534,01 FEB 2022 - 16 APR 2022,"4242 South Tamiami Trail\nVenice, FL 34293\n94...",NaN,6861,English,1088,United Way of South Sarasota County,2022-03-20T00:01:08.324409,POINT (-82.39315 27.04960)
6287,3052,Required,False,0101000020E610000067B796C970CD51C0320395F1EF63...,2020-01-10T19:32:42.11607,22 JAN 2022 - 09 APR 2022,"44 Geremonty Drive\nSalem, NH 03079\n866-444-4...",NaN,16007,English,1088,Salem High School,2022-03-20T00:01:14.039937,POINT (-71.21001 42.78076)
935,1390,Required,False,0101000020E6100000C0232A5437EB55C039F06AB933FB...,2020-01-10T19:32:28.083009,31 JAN 2022 - 21 MAR 2022,"8600 University Blvd\nEvansville, IN 47712\n81...",NaN,4473,English,1088,University of Southern Indiana,2022-03-20T00:00:56.599199,POINT (-87.67525 37.96252)
1382,2402,Not Required,False,0101000020E6100000F243A51133A254C0B3976DA7AD7D...,2020-01-10T19:32:36.559681,29 JAN 2022 - 09 APR 2022,"922 24th Street East\nBradenton, FL 34208\nVol...",SAT 8:00AM-12:00PM,25952,"English,Spanish",1088,13th Avenue Dream Center,2022-03-20T00:01:07.463982,POINT (-82.53437 27.49093)
5396,4217,Required,False,0101000020E61000006552431B80E452C023F8DF4A76E0...,2020-02-25T16:47:08.567296,05 FEB 2022 - 16 APR 2022,"2020 West 9th Street\nWilmington, DE 19805\n30...",NaN,1844,English,1088,DE-N Woodlawn Library,2022-03-20T00:01:09.93049,POINT (-75.57032 39.75361)
5381,4150,Required,False,0101000020E610000040DAFF006B4F54C02252D32EA68D...,2020-02-25T16:47:06.609762,01 FEB 2022 - 12 APR 2022,"2500 Palm Coast Parkway N W\nPalm Coast, FL 32...",NaN,7817,"Spanish,English",1088,CFWF @ Flagler County Library,2022-03-20T00:01:07.766431,POINT (-81.24091 29.55332)
5486,4139,Not Required,False,0101000020E61000001B1021AE9CA653C039D2191879E7...,2020-02-25T16:47:06.352221,29 JAN 2022 - 09 APR 2022,"2013 N Raleigh Blvd\nRaleigh, NC 27604\nVolunt...",SAT 9:00AM-2:00PM,16177,"English,Spanish",1088,Centro Para Familias Hispanas / The Hispanic F...,2022-03-20T00:01:07.380739,POINT (-78.60331 35.80838)
4301,767,Not Required,False,0101000020E6100000B75ED38382B658C001193A765049...,2020-01-10T19:32:22.714486,20 JAN 2022 - 18 APR 2022,"1904 N. Llano St\nFredericksburg, TX 78624\nVo...",MON 1:00PM-6:00PM\nTHU 1:00PM-6:00PM,24476,"English,Spanish",1088,Hill Country Community Needs Council,2022-03-20T00:00:50.23389,POINT (-98.85172 30.28638)
3836,2298,Required,False,0101000020E61000002CF015DD7A9E54C04B8FA67A3243...,2020-01-10T19:32:35.696422,01 FEB 2022 - 18 APR 2022,7227 Land OLakes Blvd.\nBuilding 8A\nLand O La...,NaN,24833,English,1088,Pasco School Board,2022-03-20T00:01:06.682193,POINT (-82.47625 28.26249)
1136,7234,Required,False,0101000020E610000068E7340BB41457C035D3BD4EEA79...,2022-02-27T00:01:26.749679,01 FEB 2022 - 12 APR 2022,"1209 E Walnut St\nColumbia, MO 65201\n573-256-...",NaN,31041,English,1088,UME MoTax Love Columbia,2022-03-20T00:00:53.096656,POINT (-92.32349 38.95246)


In [6]:
counties_gdf = geopandas.read_file("./data/tl_2019_us_county/tl_2019_us_county.shp")

In [7]:
vita_gdf = vita_gdf.set_crs(counties_gdf.crs)

### Spatial join of VITA sites and counties

Here we use [`geopandas sjoin`](https://geopandas.org/en/stable/docs/user_guide/mergingdata.html#spatial-joins) to merge the Census counties shapefile with the VITA sites. This spatial join finds all VITA sites existing within the geometry of the county.

In [8]:
vita_counties_gdf = counties_gdf.sjoin(vita_gdf)

In [9]:
vita_counties_gdf[pd.isnull(vita_counties_gdf.COUNTYFP)]

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,...,coordinates,created_at,dates,details,hours,irs_id,languages,last_scrape_id,name,updated_at


Every VITA site was matched to a county.

In [10]:
vita_counties_gdf.groupby('index_right').size().sort_values()

index_right
0       1
3276    1
3277    1
3278    1
3280    1
       ..
1474    1
1475    1
1476    1
1478    1
6297    1
Length: 3151, dtype: int64

And no VITA site appears twice. We're all set!

In [11]:
vita_counties_gdf.geometry

3       POLYGON ((-96.91075 40.78494, -96.91075 40.790...
3       POLYGON ((-96.91075 40.78494, -96.91075 40.790...
3       POLYGON ((-96.91075 40.78494, -96.91075 40.790...
3       POLYGON ((-96.91075 40.78494, -96.91075 40.790...
3       POLYGON ((-96.91075 40.78494, -96.91075 40.790...
                              ...                        
3217    POLYGON ((-72.95460 42.62865, -72.95104 42.641...
3227    POLYGON ((-84.36198 41.89876, -84.36198 41.898...
3227    POLYGON ((-84.36198 41.89876, -84.36198 41.898...
3230    POLYGON ((-90.09363 32.70763, -90.09360 32.707...
3232    POLYGON ((-82.59529 38.36978, -82.59515 38.369...
Name: geometry, Length: 3151, dtype: geometry

In [29]:
# Lat/long of VITA sites for Tableau
vita_counties_gdf['lat_long'] = geopandas.GeoSeries.from_wkb(vita_counties_gdf.coordinates)
vita_counties_gdf['longitude'] = vita_counties_gdf.lat_long.apply(lambda point: point.x)
vita_counties_gdf['latitude'] = vita_counties_gdf.lat_long.apply(lambda point: point.y)
vita_counties_gdf.drop(columns=['lat_long'], inplace=True)

In [ ]:
vita_counties_gdf.to_file("vita_sites_by_county.shp")

In [30]:
pd.DataFrame(vita_counties_gdf).to_csv("vita_sites_by_county.csv", index=False)

In [25]:
vita_counties_gdf['lat_long'].head()

3    POINT (-96.70375 40.80471)
3    POINT (-96.70108 40.81320)
3    POINT (-96.59839 40.81336)
3    POINT (-96.65845 40.81392)
3    POINT (-96.70102 40.81689)
Name: lat_long, dtype: geometry